In [1]:
import os
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from dotenv import load_dotenv

from vectordb import SimilarProductVectorDB

load_dotenv()

True

# Load data

In [2]:
# read product data
spark = SparkSession.builder.appName("read_data").getOrCreate()
data = spark.read.parquet(os.environ["PRODUCT_DATA_PATH"])

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/01 14:39:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- category: string (nullable = true)
 |-- class: string (nullable = true)
 |-- subclass: string (nullable = true)
 |-- product_description: string (nullable = true)



# Initialize vectorstore

In [4]:
vectorstore = SimilarProductVectorDB(
    collection_name="cosmetics_similar_product_db",
    distance_function="ip",
    n_query_result=10,  # show at most top 10 most similar product in result
)

In [5]:
# extract documents and metadata from table
df = data.toPandas()
# adding random stock level
df["stock_level"] = np.random.randint(0, 101, size=len(df))

In [6]:
df.head()

,product_id,product_name,brand,category,class,subclass,product_description,stock_level
0,KLK169,Luminous Deep Hydration Lifting Mask,TATCHA,Skincare,Masks & Treatments,Sheet Mask,Boosting moisture levels up to 200% in 15 minu...,9
1,LGY307,Jour d'Hermès Absolu Eau de Parfum 50ml,HERMÈS,Fragrance,Perfume,Spray,I wanted to express the essence of femininity ...,75
2,LGY220,Jour d'Hermès Absolu Eau de Parfum 85ml,HERMÈS,Fragrance,Perfume,Spray,I wanted to express the essence of femininity ...,40
3,LKK287,Love In White Spray 75ml,CREED,Fragrance,Perfume,Spray,"Inspired by a love for sailing, this Millésime...",8
4,AKY054,Neroli Portofino Eau de Parfum 100ml,TOM FORD,Fragrance,Perfume,Spray,Vibrant. Sparkling. Transportive.<br>To TOM FO...,20


# Add to vectorstore
Simply use product description as the features and pass into the vectorstore.

In [7]:
# put document into vectordb
product_ids = list(df["product_id"])
product_desc = list(df["product_description"])
product_metadatas = df[["product_name", "brand", "category", "class", "subclass", "stock_level"]].to_dict(
    orient='records')

vectorstore.add_documents(
    documents=product_desc,
    metadatas=product_metadatas,
    product_ids=product_ids,
)

# Query

In [13]:
# small function for visualizing the result
def visualize_result(query_result):
    result_dict = query_result["metadatas"][0]
    query_id = query_result["ids"][0][0]
    for i, (_id, distance) in enumerate(zip(query_result["ids"][0], query_result["distances"][0])):
        result_dict[i]["product_id"] = _id
        result_dict[i]["distance"] = round(distance, 3)
    result_df = pd.DataFrame(result_dict)
    result_df = result_df[["product_id", "product_name", "class", "subclass", "stock_level", "brand", "distance"]]
    print(f"For Product {query_id}, the top {len(query_result['ids'][0]) - 1} most similar items are:")
    display(result_df)

In [14]:
random_product_id = np.random.choice(product_ids, 1)[0]
random_product_id

'AKW566'

In [15]:
result = vectorstore.query_with_product_id(random_product_id)
result

{'ids': [['AKW566',
   'AKW568',
   'AKW565',
   'AKW569',
   'AJB605',
   'AKW563',
   'AJB603',
   'AJB609',
   'AJB602',
   'ALB647']],
 'distances': [[1.7881393432617188e-07,
   0.06979924440383911,
   0.07616376876831055,
   0.08861368894577026,
   0.16402119398117065,
   0.2724398970603943,
   0.27301663160324097,
   0.27621912956237793,
   0.2779233455657959,
   0.2978266477584839]],
 'metadatas': [[{'brand': 'FRESH',
    'category': 'Skincare',
    'class': 'Lip Care',
    'product_name': 'Sugar Peach Hydrating Lip Balm Limited Edition',
    'stock_level': 33,
    'subclass': 'Lip Balm'},
   {'brand': 'FRESH',
    'category': 'Skincare',
    'class': 'Lip Care',
    'product_name': 'Sugar Chocolate Hydrating Lip Balm Limited Edition',
    'stock_level': 89,
    'subclass': 'Lip Balm'},
   {'brand': 'FRESH',
    'category': 'Skincare',
    'class': 'Lip Care',
    'product_name': 'Sugar Lemon Hydrating Lip Balm Limited Edition',
    'stock_level': 28,
    'subclass': 'Lip Balm'}

In [17]:
visualize_result(result)

For Product AKW566, the top 9 most similar items are:


,product_id,product_name,class,subclass,stock_level,brand,distance
0,AKW566,Sugar Peach Hydrating Lip Balm Limited Edition,Lip Care,Lip Balm,33,FRESH,0.000
1,AKW568,Sugar Chocolate Hydrating Lip Balm Limited Edi...,Lip Care,Lip Balm,89,FRESH,0.070
2,AKW565,Sugar Lemon Hydrating Lip Balm Limited Edition,Lip Care,Lip Balm,28,FRESH,0.076
3,AKW569,Sugar Coconut Hydrating Lip Balm,Lip Care,Lip Balm,60,FRESH,0.089
4,AJB605,Sugar Lip Caramel Hydrating Balm,Lip Care,Lip Balm,1,FRESH,0.164
5,AKW563,Sugar Dream Lip Treatment Advanced Therapy,Lip Care,Lip Treatment,99,FRESH,0.272
6,AJB603,Sugar Cream Lip Treatment – Baby,Lip Care,Lip Treatment,42,FRESH,0.273
7,AJB609,Sugar Cream Lip Treatment – Gilt,Lip Care,Lip Treatment,5,FRESH,0.276
8,AJB602,Sugar Cream Lip Treatment – Pearl,Lip Care,Lip Treatment,13,FRESH,0.278
9,ALB647,Rose Petal Lip Balm 4.4g,Lip Care,Lip Balm,47,SUBTLE ENERGIES,0.298


# Use Extracted Features
Use extracted features as the features and pass the embeddings into the vectorstore.

In [18]:
# load data
data = spark.read.parquet(os.environ["PERFUME_PRODUCT_DATA_PATH"])
data.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- category: string (nullable = true)
 |-- class: string (nullable = true)
 |-- subclass: string (nullable = true)
 |-- product_description: string (nullable = true)
 |-- features: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [19]:
# another vectorstore for features
perfume_vectorstore = SimilarProductVectorDB(
    collection_name="perfume_similar_product_db",
    distance_function="ip",
    n_query_result=10,  # show at most top 10 most similar product in result
)

In [20]:
# extract documents and metadata from table
df = data.toPandas()
# adding random stock level
df["stock_level"] = np.random.randint(0, 101, size=len(df))
df.head()

,product_id,product_name,brand,category,class,subclass,product_description,features,stock_level
0,YBB493,Acqua Di Gioia Eau de Parfum 100ml,GIORGIO ARMANI BEAUTY,Fragrance,Perfume,Spray,"Perfect for the spring and summer, Giorgio Arm...","[lmr cedarwood heart, brown sugar, labdanum]",15
1,YQX798,My Burberry Eau De Parfum 90ml,BURBERRY BEAUTY,Fragrance,Perfume,Spray,<li>90ml</li><li>Contemporary floral scent</li...,"[sweet pea, bergamot fuse, geranium leaf, gold...",69
2,OYK369,Rose Amazone Eau de Toilette 100ml,HERMÈS,Fragrance,Perfume,Spray,A new Amazone who is so much more contemporary...,"[citrus fruits, currant, raspberry]",42
3,JIK820,Arancia di Capri Eau de Toilette 150ml,ACQUA DI PARMA,Fragrance,Perfume,Spray,Reminisce on the summer vacations at Capri wit...,"[orange, mandarin, lemon, petitgrain, cardamon...",39
4,EOI621,Blu Mediterraneo Fico Di Amalfi Eau de Toilett...,ACQUA DI PARMA,Fragrance,Perfume,Spray,"A unique combination of fig nectar, jasmine an...","[bergamot, lemon, grapefruit, citron, fig nect...",10


In [21]:
# embed the lists of words
from sklearn.preprocessing import MultiLabelBinarizer

# extract the complete vocabulary of unique words
vocabulary = set(word for sublist in df["features"] for word in sublist)

# initialize MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=list(vocabulary))

# fit and transform the 'features' column
word_freq_vectors = mlb.fit_transform(df['features'])
words = mlb.classes_

# normalize the vector
normalized_word_freq_array = word_freq_vectors / np.linalg.norm(word_freq_vectors, axis=1, keepdims=True)

In [22]:
# put embedding into vectordb
product_ids = list(df["product_id"])
product_metadatas = df[["product_name", "brand", "category", "class", "subclass", "stock_level"]].to_dict(
    orient='records')

perfume_vectorstore.add_embeddings(
    embeddings=normalized_word_freq_array.tolist(),
    metadatas=product_metadatas,
    product_ids=product_ids,
)

In [23]:
visualize_result(perfume_vectorstore.query_with_product_id(np.random.choice(product_ids, 1)[0]))

For Product EHL515, the top 9 most similar items are:


,product_id,product_name,class,subclass,stock_level,brand,distance
0,EHL515,Papier Carbone Eau de Parfum 20ml,Perfume,Spray,60,ORMAIE,0.000
1,EHL544,Papier Carbone Eau de Parfum 50ml,Perfume,Spray,49,ORMAIE,0.000
2,EHL492,Papier Carbone Eau de Parfum 100ml,Perfume,Spray,25,ORMAIE,0.000
3,EHL540,Papier Carbone The Ampoule Refill 20ml,Perfume,Spray,6,ORMAIE,0.000
4,BWO448,PRIVÉ VÉTIVER D'HIVER Eau de Toilette 50ML,Perfume,Spray,41,GIORGIO ARMANI BEAUTY,0.625
5,UGI240,Sunday Cologne Eau De Parfum 50ml,Perfume,Spray,97,BYREDO,0.646
6,UGI237,Sunday Cologne Eau de Parfum 100ml,Perfume,Spray,24,BYREDO,0.646
7,BQW013,Vetiver eau de parfum 50ml,Perfume,Spray,87,MALIN+GOETZ,0.711
8,EHL509,Le Passant Eau de Parfum 20ml,Perfume,Spray,70,ORMAIE,0.733
9,EHL494,Le Passant Eau de Parfum 50ml,Perfume,Spray,100,ORMAIE,0.733
